# Hospital Facilities Insight 

Data obtained from Community Benefits Insight: http://www.communitybenefitinsight.org/api/get_hospitals.php

## Import Libaries

In [1]:
# import appropriate packages
import os
import requests
import json 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql.cursors

import warnings
warnings.filterwarnings("ignore") #supress warnings

/var/folders/7x/mfyhhlkn0fz774bmn50_szsm0000gn/T/ipykernel_85623/4251200241.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Import Data

In [2]:
# API 
api_url = 'http://www.communitybenefitinsight.org/api/get_hospitals.php'

# request for api 
response = requests.get(api_url)

# check status
if response.status_code == 200:
    data = response.json()
    
    # data update check 
    last_modified_header = response.headers.get('Last-Modified')
    
    if last_modified_header:
        last_modified = datetime.strptime(last_modified_header, '%a, %d %b %Y %H:%M:%S GMT')
        print(f"Data was last modified: {last_modified}")

    # convert data to df 
    hospital_df = pd.DataFrame(data)
    
    print(hospital_df)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(response.text)

     hospital_id hospital_org_id        ein  \
0              1               1  630307951   
1              2               2  630578923   
2              3               3  630312913   
3              4               4  630459034   
4              5               5  581973570   
...          ...             ...        ...   
3486        3487            2647  813040663   
3487        3488            2304  741109643   
3488        3489            2648  831954982   
3489        3490            2302  750800661   
3490        3491            2649  831869297   

                                     name  \
0                Mizell Memorial Hospital   
1                        St Vincents East   
2           Shelby Baptist Medical Center   
3            Callahan Eye Foundation Hosp   
4                 Cherokee Medical Center   
...                                   ...   
3486          Bsw Medical Center - Austin   
3487              Ascension Seton Bastrop   
3488         Texas Health Hosp

In [3]:
hospital_df # print df 

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
0,1,1,630307951,Mizell Memorial Hospital,Mizell Memorial Hospital,702 Main Street,Opp,AL,36462,01039,99,N,N,N,N,010007,Covington County,<100 beds,"November 20, 2023"
1,2,2,630578923,St Vincents East,St Vincents East,50 Medical Park Drive East,Birmingham,AL,35235,01073,362,N,Y,N,Y,010011,Jefferson County,>299 beds,"November 20, 2023"
2,3,3,630312913,Shelby Baptist Medical Center,Shelby Baptist Medical Center,1000 First Street North,Alabaster,AL,35007,01117,252,N,Y,N,N,010016,Shelby County,100-299 beds,"November 20, 2023"
3,4,4,630459034,Callahan Eye Foundation Hosp,Callahan Eye Foundation Hosp,1720 University Boulevard,Birmingham,AL,35233,01073,106,N,Y,N,Y,010018,Jefferson County,100-299 beds,"November 20, 2023"
4,5,5,581973570,Cherokee Medical Center,Cherokee Medical Center,400 Northwood Drive,Centre,AL,35960,01019,60,N,N,N,N,010022,Cherokee County,<100 beds,"November 20, 2023"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,3487,2647,813040663,Bsw Medical Center - Austin,Bsw Medical Center - Austin,5245 W Us 290,Austin,TX,78735,48453,16,N,Y,N,N,670136,Travis County,<100 beds,"November 20, 2023"
3487,3488,2304,741109643,Ascension Seton Bastrop,Ascension Seton Bastrop,630 Highway 71 W,Bastrop,TX,78602,48021,7,N,Y,N,N,670143,Bastrop County,<100 beds,"November 20, 2023"
3488,3489,2648,831954982,Texas Health Hospital Frisco,Texas Health Hospital Frisco,12400 N Dallas Parkway,Frisco,TX,75033,48121,63,N,Y,N,N,670260,Denton County,<100 beds,"November 20, 2023"
3489,3490,2302,750800661,Methodist Midlothian Medical Center,Methodist Midlothian Medical Center,1201 E Highway 287,Midlothian,TX,76065,48139,46,N,Y,N,N,670300,Ellis County,<100 beds,"November 20, 2023"


In [4]:
# print 10 random samples from the dataframe 
hospital_df.sample(10)

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
838,839,611,362596381,Alexian Brothers Medical Center,Alexian Brothers Medical Center,800 Biesterfield Road,Elk Grove Village,IL,60007,17031,483,N,Y,N,N,140258,Cook County,>299 beds,"November 20, 2023"
2599,2600,2053,231352222,York Hospital,York Hospital,1001 South George Street,York,PA,17405,42133,572,N,Y,N,Y,390046,York County,>299 beds,"November 20, 2023"
257,258,183,941450151,Marshall Hospital,Marshall Hospital,1100 Marshall Way,Placerville,CA,95667,06017,120,N,Y,N,N,050254,El Dorado County,100-299 beds,"November 20, 2023"
288,289,136,951683892,Cmh Of San Buenaventura,Cmh Of San Buenaventura,147 N Brent Street,Ventura,CA,93003,06111,242,N,Y,N,Y,050394,Ventura County,100-299 beds,"November 20, 2023"
2482,2483,1964,731430157,Craig General Hospital,Craig General Hospital,735 North Foreman,Vinita,OK,74301,40035,62,N,N,N,N,370065,Craig County,<100 beds,"November 20, 2023"
1479,1480,1105,381459362,William Beaumont Hospital - Troy,William Beaumont Hospital - Troy,44201 Dequindre,Troy,MI,48098,26125,254,N,Y,N,Y,230269,Oakland County,100-299 beds,"November 20, 2023"
674,675,483,582322328,Piedmont Fayette Hospital Inc,Piedmont Fayette Hospital Inc,1255 Highway 54 West,Fayetteville,GA,30214,13113,221,N,Y,N,N,110215,Fayette County,100-299 beds,"November 20, 2023"
2276,2277,1797,450308379,Sakakawea Medical Center,Sakakawea Medical Center,510 J8th Ave Ne,Hazen,ND,58545,38057,13,N,N,N,N,351310,Mercer County,<100 beds,"November 20, 2023"
1041,1042,780,480581968,Susan B Allen Memorial Hospital,Susan B Allen Memorial Hospital,720 W Central,El Dorado,KS,67042,20015,80,N,Y,N,N,170017,Butler County,<100 beds,"November 20, 2023"
3375,3376,2537,611649250,Aurora Medical Center Grafton,Aurora Medical Center Grafton,975 Port Washington Road,Grafton,WI,53024,55089,62,N,Y,N,N,520207,Ozaukee County,<100 beds,"November 20, 2023"


## Data Inspection

In [5]:
hospital_df.shape 

(3491, 19)

In [6]:
hospital_df.info() #all variables in df are labeled as object variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   hospital_id                 3491 non-null   object
 1   hospital_org_id             3491 non-null   object
 2   ein                         3491 non-null   object
 3   name                        3491 non-null   object
 4   name_cr                     3491 non-null   object
 5   street_address              3491 non-null   object
 6   city                        3491 non-null   object
 7   state                       3491 non-null   object
 8   zip_code                    3491 non-null   object
 9   fips_state_and_county_code  3491 non-null   object
 10  hospital_bed_count          3491 non-null   object
 11  chrch_affl_f                3491 non-null   object
 12  urban_location_f            3491 non-null   object
 13  children_hospital_f         3491 non-null   obje

In [7]:
columns_to_convert = ['hospital_bed_count', 'medicare_provider_number']
binary_columns = ['chrch_affl_f', 'urban_location_f', 'children_hospital_f', 'memb_counc_teach_hosps_f']

# iterate through the list of columns
for col in columns_to_convert:
    # convert to numeric, coerce errors to NaN
    hospital_df[col] = pd.to_numeric(hospital_df[col], errors='coerce')
    
for col in binary_columns:
    # Map 'N' to 0 and 'Y' to 1
    hospital_df[col] = hospital_df[col].map({'N': 0, 'Y': 1})

# check the data types after conversion
hospital_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   hospital_id                 3491 non-null   object
 1   hospital_org_id             3491 non-null   object
 2   ein                         3491 non-null   object
 3   name                        3491 non-null   object
 4   name_cr                     3491 non-null   object
 5   street_address              3491 non-null   object
 6   city                        3491 non-null   object
 7   state                       3491 non-null   object
 8   zip_code                    3491 non-null   object
 9   fips_state_and_county_code  3491 non-null   object
 10  hospital_bed_count          3491 non-null   int64 
 11  chrch_affl_f                3491 non-null   int64 
 12  urban_location_f            3491 non-null   int64 
 13  children_hospital_f         3491 non-null   int6

In [8]:
hospital_df.nunique() #unique variables 

hospital_id                   3491
hospital_org_id               2377
ein                           2377
name                          3330
name_cr                       3072
street_address                3439
city                          2192
state                           51
zip_code                      3200
fips_state_and_county_code    1751
hospital_bed_count             696
chrch_affl_f                     2
urban_location_f                 2
children_hospital_f              2
memb_counc_teach_hosps_f         2
medicare_provider_number      3491
county                        1195
hospital_bed_size                3
updated_dt                       1
dtype: int64

In [9]:
hospital_df.isnull().sum() #null values 

hospital_id                   0
hospital_org_id               0
ein                           0
name                          0
name_cr                       0
street_address                0
city                          0
state                         0
zip_code                      0
fips_state_and_county_code    0
hospital_bed_count            0
chrch_affl_f                  0
urban_location_f              0
children_hospital_f           0
memb_counc_teach_hosps_f      0
medicare_provider_number      0
county                        1
hospital_bed_size             0
updated_dt                    0
dtype: int64

In [10]:
duplicate_rows = hospital_df[hospital_df.duplicated()]

print("Duplicate Rows except first occurrence:")
print(duplicate_rows)

if hospital_df.duplicated().any():
    print("Duplicates exist in the DataFrame.")
else:
    print("No duplicates found in the DataFrame.")

Duplicate Rows except first occurrence:
Empty DataFrame
Columns: [hospital_id, hospital_org_id, ein, name, name_cr, street_address, city, state, zip_code, fips_state_and_county_code, hospital_bed_count, chrch_affl_f, urban_location_f, children_hospital_f, memb_counc_teach_hosps_f, medicare_provider_number, county, hospital_bed_size, updated_dt]
Index: []
No duplicates found in the DataFrame.


## Table Manipulation

In [11]:
# Hospital Information Table

# create unique ID for address
hospital_df['address_id'] = hospital_df['street_address'] + '_' + hospital_df['city'] + '_' + hospital_df['state'] + '_' + hospital_df['zip_code'] + '_' + hospital_df['fips_state_and_county_code']
hospital_df['address_id'] = hospital_df['address_id'].rank(method='dense', ascending=False).astype(int)

#create unique ID for hospital capacity
hospital_df['capacity_id'] = hospital_df['hospital_id'].astype(str) + '_' + hospital_df['hospital_bed_count'].astype(str) + '_' + hospital_df['hospital_bed_size']
hospital_df['capacity_id'] = hospital_df['capacity_id'].rank(method='dense', ascending=False).astype(int)


# create hospital information table
hospital = hospital_df[['hospital_id', 'hospital_org_id', 'name', 'ein', 'medicare_provider_number', 'address_id', 'capacity_id', 'updated_dt']]
hospital

,hospital_id,hospital_org_id,name,ein,medicare_provider_number,address_id,capacity_id,updated_dt
0,1,1,Mizell Memorial Hospital,630307951,10007,609,2381,2023-11-20
1,2,2,St Vincents East,630578923,10011,1075,1270,2023-11-20
2,3,3,Shelby Baptist Medical Center,630312913,10016,3323,667,2023-11-20
3,4,4,Callahan Eye Foundation Hosp,630459034,10018,2440,556,2023-11-20
4,5,5,Cherokee Medical Center,581973570,10022,1337,445,2023-11-20
...,...,...,...,...,...,...,...,...
3486,3487,2647,Bsw Medical Center - Austin,813040663,670136,949,730,2023-11-20
3487,3488,2304,Ascension Seton Bastrop,741109643,670143,718,729,2023-11-20
3488,3489,2648,Texas Health Hospital Frisco,831954982,670260,2901,728,2023-11-20
3489,3490,2302,Methodist Midlothian Medical Center,750800661,670300,2967,726,2023-11-20


In [12]:
# Church Affiliation Table
church_affiliation = hospital_df[['hospital_id', 'chrch_affl_f', 'updated_dt']]
church_affiliation

,hospital_id,chrch_affl_f,updated_dt
0,1,0,2023-11-20
1,2,0,2023-11-20
2,3,0,2023-11-20
3,4,0,2023-11-20
4,5,0,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [13]:
# Hospital Capacity Table extracting data from hospital_df
hospital_capacity = hospital_df[['capacity_id', 'hospital_bed_size', 'hospital_bed_count', 'updated_dt']]
hospital_capacity

,capacity_id,hospital_bed_size,hospital_bed_count,updated_dt
0,2381,<100 beds,99,2023-11-20
1,1270,>299 beds,362,2023-11-20
2,667,100-299 beds,252,2023-11-20
3,556,100-299 beds,106,2023-11-20
4,445,<100 beds,60,2023-11-20
...,...,...,...,...
3486,730,<100 beds,16,2023-11-20
3487,729,<100 beds,7,2023-11-20
3488,728,<100 beds,63,2023-11-20
3489,726,<100 beds,46,2023-11-20


In [14]:
# Children Hospital Table - extracting data from hospital_df
children_hospital= hospital_df[['hospital_id', 'children_hospital_f', 'updated_dt']]
children_hospital

,hospital_id,children_hospital_f,updated_dt
0,1,0,2023-11-20
1,2,0,2023-11-20
2,3,0,2023-11-20
3,4,0,2023-11-20
4,5,0,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [15]:
# Teaching Table - extracting data from hospital_df
teaching = hospital_df[['hospital_id', 'memb_counc_teach_hosps_f', 'updated_dt']]
teaching

,hospital_id,memb_counc_teach_hosps_f,updated_dt
0,1,0,2023-11-20
1,2,1,2023-11-20
2,3,0,2023-11-20
3,4,1,2023-11-20
4,5,0,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [16]:
# Urban Table
urban = hospital_df[['urban_location_f', 'updated_dt','hospital_id']]
urban

,urban_location_f,updated_dt,hospital_id
0,0,2023-11-20,1
1,1,2023-11-20,2
2,1,2023-11-20,3
3,1,2023-11-20,4
4,0,2023-11-20,5
...,...,...,...
3486,1,2023-11-20,3487
3487,1,2023-11-20,3488
3488,1,2023-11-20,3489
3489,1,2023-11-20,3490


In [17]:
# connect to database

username = "jeremiahf24"
password = "$ads507password"
hostName = "ads507finalproject.mysql.database.azure.com"

conn = pymysql.connect(host=hostName,
                       port=3306,
                       user=username,
                       passwd=password, 
                       db= "ads507final", 
                       ssl={"fake_flag_to_enable_tls": True})

In [18]:
tableNames=pd.read_sql_query("""SHOW TABLES""", conn)

# print table names 
tableNames

,Tables_in_ads507final
0,address_location
1,children_hospital
2,church_affiliation
3,hospital
4,hospital_capacity
5,teaching
6,urban


In [24]:
# children hospital table - convert df to list of tuples
children_hospital_data = [tuple(x) for x in children_hospital.values]

# SQL query - insert data into children_hospital table
sql_insert_children_hospital = """
INSERT INTO children_hospital(hospital_id, children_hospital_f, updated_dt)
VALUES (%s, %s, %s)
"""

# execute query 
cursor = conn.cursor()
cursor.executemany(sql_insert_children_hospital, children_hospital_data)
conn.commit()

In [21]:
# hospital_capacity table - convert df to list of tuples
hospital_capacity_data = [tuple(x) for x in hospital_capacity.values]

# SQL query - insert data into hospital_capacity table
sql_insert_hospital_capacity = """
INSERT INTO hospital_capacity (capacity_id, hospital_bed_size, hospital_bed_count, updated_dt)
VALUES (%s, %s, %s, %s)
"""

# execute query 
cursor = conn.cursor()
cursor.executemany(sql_insert_hospital_capacity, hospital_capacity_data)
conn.commit()

In [28]:
# teaching table - convert df to list of tuples
teaching_data = [tuple(x) for x in teaching.values]

# SQL query - insert data into children_hospital table
sql_insert_teaching = """
INSERT INTO teaching (hospital_id, memb_counc_teach_hosps_f, updated_dt)
VALUES (%s, %s, %s)
"""

# execute query 
cursor = conn.cursor()
cursor.executemany(sql_insert_teaching, teaching_data)
conn.commit()

In [29]:
# Create a cursor
cursor = conn.cursor()

# Execute SQL queries to get table names
cursor.execute("SHOW TABLES")

# Fetch all the tables
tables = cursor.fetchall()

for table in tables:
        table_name = table[0]

        # Execute SQL query to fetch sample data from each table
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 5")

        # Fetch and print sample data
        sample_data = cursor.fetchall()
        print(f"Sample data from table '{table_name}':")
        for row in sample_data:
            print(row)

Sample data from table 'address_location':
Sample data from table 'children_hospital':
(1, 0, '2023-11-20')
(2, 0, '2023-11-20')
(3, 0, '2023-11-20')
(4, 0, '2023-11-20')
(5, 0, '2023-11-20')
Sample data from table 'church_affiliation':
Sample data from table 'hospital':
Sample data from table 'hospital_capacity':
(1, '<100 beds', 69, '2023-11-20')
(2, '<100 beds', 25, '2023-11-20')
(3, '100-299 beds', 234, '2023-11-20')
(4, '100-299 beds', 150, '2023-11-20')
(5, '>299 beds', 649, '2023-11-20')
Sample data from table 'teaching':
(1, 0, '2023-11-20')
(2, 1, '2023-11-20')
(3, 0, '2023-11-20')
(4, 1, '2023-11-20')
(5, 0, '2023-11-20')
Sample data from table 'urban':
